In [ ]:
# if hit plotting library issues, try resetting LD path for julia
# can set in ~/.local/share/jupyter/kernels/
# @assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
pkgs = [
    "Revise",
    # "FASTX",
    # "StatsBase",
    # "Distributions",
    # "StatsPlots",
    # "Random",
    # "Dates",
    # "DataFrames",
    # "BioSequences",
    "Conda",
    # "Downloads"
]
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
# Pkg.develop(path="/global/cfs/projectdirs/m4269/cjprybol/Mycelia")
# Pkg.develop(path="../../..")
import Mycelia

In [ ]:
# run(`$(Mycelia.MAMBA) clean --all -y`)
# run(`$(Mycelia.MAMBA) remove --name clair3-illumina --all`)

In [ ]:
# Pkg.build("Mycelia")

In [ ]:
PROJECT_BASEDIR = dirname(pwd())
data_dir = joinpath(PROJECT_BASEDIR, "data")
genome_dir = mkpath(joinpath(data_dir, "genomes"))

In [ ]:
bams = sort(filter(x -> occursin(r"\.sorted\.bam$", x), readdir(genome_dir, join=true)), by=x->filesize(x))

In [ ]:
threads = 1
for bam in bams
    if !isfile(bam * ".bai")
        run(`$(Mycelia.MAMBA) run --live-stream -n samtools samtools index $(bam)`)
    end
    outdir = bam * ".clair3"
    outfile = joinpath(outdir, "merge_output.vcf.gz")
    # output = bam * ".clair3.vcf"
    fasta_reference = replace(bam, r"\.normalized\.vcf\.fna.*" => "")
    if !isfile(outfile) || (filesize(outfile) == 0)
        if occursin(".art.", bam)
            cmd = 
            """
            $(Mycelia.MAMBA) run --live-stream -n clair3 run_clair3.sh --haploid_precise --no_phasing_for_fa --include_all_ctgs --platform="ilmn" --model_path="$(Conda.prefix(Symbol("clair3-illumina")))/bin/models/ilmn" --bam_fn=$(bam) --ref_fn=$(fasta_reference) --threads=$(threads) --output=$(outdir)
            """
            # cmd = `$(Mycelia.MAMBA) run --live-stream -n clair3-illumina run_clair3.sh --haploid_precise --no_phasing_for_fa --include_all_ctgs --platform="ilmn" --model_path="$(Conda.prefix(Symbol("clair3-illumina")))/bin/models/ilmn" --bam_fn=$(bam) --ref_fn=$(fasta_reference) --threads=$(threads) --output=$(output)`
        elseif occursin(".badread.", bam)
            cmd = 
            """
            $(Mycelia.MAMBA) run --live-stream -n clair3 run_clair3.sh --haploid_precise --no_phasing_for_fa --include_all_ctgs --platform="ont" --model_path="$(Conda.prefix(:clair3))/bin/models/r941_prom_sup_g5014" --bam_fn=$(bam) --ref_fn=$(fasta_reference) --threads=$(threads) --output=$(outdir)
            """
            # cmd = `$(Mycelia.MAMBA) run --live-stream -n clair3 run_clair3.sh run_clair3.sh --haploid_precise --no_phasing_for_fa --include_all_ctgs --platform="ont" --model_path="$(Conda.prefix(:clair3))/bin/models/r941_prom_sup_g5014" --bam_fn=$(bam) --ref_fn=$(fasta_reference) --threads=$(threads) --output=$(output)`
        else
            error()
        end
    # run(cmd)
    Mycelia.nersc_sbatch(
        job_name = "$(basename(outdir))",
        mail_user = "cameron.prybol@gmail.com",
        logdir = mkpath("$(homedir())/workspace/slurmlogs"),
        qos = "shared",
        cpus_per_task= threads,
        cmd = cmd)
    else
        @info "$(outfile) already present..."
    end
end